In [ ]:
'''Project description
Creating a program to pull up country data from variable sources

current status:
1. managed to query WB data for a user specified country
2. Create a user interface

To-do list:
1. Pull in more informaion for other sources
2. Include another country in the query as benchmark (for user to make sense of the data)
3. Improve the user interface
'''

In [2]:
##### Prep-->package installation

!pip install wbpy
import wbpy
from pprint import pprint

!pip install pycountry
import pycountry

In [3]:
#### Prep-->define indicator list

#https://github.com/mattduck/wbpy
api = wbpy.IndicatorAPI()

# Indicator list

#######  low freq
#GDP (real, Nominal; level and growth rates)
ngdp='ny.gdp.mktp.cd' #usd
cgdp='NY.GDP.MKTP.KD' #2010 usd
#Population
total_population = "SP.POP.TOTL"
#Current account balance (USD)
cab='BN.CAB.XOKA.CD'
#Unemployment - %of total labor force
unemployment='SL.UEM.TOTL.NE.ZS'
#Doing business index (1=most friendly)
dbi='IC.BUS.EASE.XQ'


# Combined indicator list
i_list=[ngdp,cgdp,total_population,cab,unemployment,dbi]
n_list=['Nominal GDP in USD','Real GDP in 2010 USD','Total population','Current account balance in USD','Unemployment rate','Doing business index (1=most friendly)']

In [4]:
from tkinter import *
top_widget=Tk()
top_widget.geometry('500x400+650+300')
top_widget.title('Find country info')

label1=Label(top_widget,text='Enter a country name',bg='gold',fg='blue').place(x=25,y=25)

label3=Label(top_widget,text='Output',bg='white',fg='blue')
label3.place(x=25,y=85)
label3.config(width=40)

label4=Label(top_widget,text='Warnings',bg='white',fg='blue')
label4.place(x=25,y=265)

box1=Entry(top_widget)


box1.focus()

box1.place(x=160,y=25)

    
def get_cntydata():
    c_matched=pycountry.countries.search_fuzzy(box1.get())  #fuzzy match user input of a country to country codes
    match=c_matched[0]      
    c_code=match.alpha_2
    flag=''
    
    result={}
    for item in i_list:
        item=api.get_dataset(item, [c_code], date="2018:2020")
        #         query the last three years and keep the most recent number; if no values returned, print "no updates since 2017"
        result[item]=item.as_dict()               #complete the API query by country
    #print(result) 
    
    result1={}
    for n in n_list:
        result1[n]=result1.get(n,0)
    
    result2={} #         restructure the returned results
    for code,item,name in zip(i_list,result.values(),n_list):
        for a in item.values():
            result2['2020']=a['2020']
            result2['2019']=a['2019']
            result2['2018']=a['2018']
            if result2['2020'] is None:
                if result2['2019'] is None:
                    if result2['2018'] is None:
                        result1[name]=0
                        flag="Warning: Variable "+name+ " has no updates since 2017"
                    else:
                        result1[name]=result2['2018']
                else:
                    result1[name]=result2['2019']
            else:
                result1[name]=result2['2020'] 
    #label3.config(text=[result1,flag])
    
    label3['text'] = '\n'.join('{} {}'.format(k, d) for k, d in result1.items())
    label4.config(text=flag)
    
btn=Button(top_widget,text='Get country information',command=get_cntydata).place(x=170,y=55)
top_widget.mainloop()

http://api.worldbank.org/v2/countries/CHN/indicators/ny.gdp.mktp.cd?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN/indicators/NY.GDP.MKTP.KD?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN/indicators/SP.POP.TOTL?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN/indicators/BN.CAB.XOKA.CD?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN/indicators/SL.UEM.TOTL.NE.ZS?date=2018%3A2020&format=json&per_page=10000
http://api.worldbank.org/v2/countries/CHN/indicators/IC.BUS.EASE.XQ?date=2018%3A2020&format=json&per_page=10000
